In [16]:
from utils import (
    TransactionDataset,
    TuneHyperParams,
    FinalModelPerformance
)

from constants import Columns, ModelConstants, ResamplingStrategy

In [23]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import (
    OneHotEncoder,
    OrdinalEncoder,
    FunctionTransformer,
    Normalizer,
    PowerTransformer,
    RobustScaler
)
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.compose import make_column_transformer, make_column_selector

from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE, ADASYN
from imblearn.under_sampling import TomekLinks, NearMiss

from joblib import dump


In [18]:
data = TransactionDataset().get_training_test_split()

In [19]:
log_transform = FunctionTransformer(
    func=np.log1p, inverse_func=np.expm1, check_inverse=False
)

order_mapping = [['0', '0.2', '0.4', '0.6', '0.8', '1'], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]

norm_and_one_hot_transforms = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(categories=order_mapping), [Columns.SPECIFIC_HOLIDAY, Columns.MONTH]),
    ("drop", [Columns.INFO_PAGE_TIME, Columns.PRODUCT_PAGE_TIME, Columns.GOOGLE_ANALYTICS_ER]),
    (Normalizer(), make_column_selector(dtype_include="number"))
)

log_and_ordinal_transforms = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(categories=order_mapping), [Columns.SPECIFIC_HOLIDAY, Columns.MONTH]),
    ("drop", [Columns.INFO_PAGE_TIME, Columns.PRODUCT_PAGE_TIME, Columns.GOOGLE_ANALYTICS_ER]),
    (log_transform, make_column_selector(dtype_include="number"))
)

power_transforms = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(categories=order_mapping), [Columns.SPECIFIC_HOLIDAY, Columns.MONTH]),
    ("drop", [Columns.INFO_PAGE_TIME, Columns.PRODUCT_PAGE_TIME, Columns.GOOGLE_ANALYTICS_ER]),
    (PowerTransformer(), make_column_selector(dtype_include="number"))
)

robust_transformation = make_column_transformer(
    (OneHotEncoder(), [Columns.CUSTOMER_TYPE]),
    (OrdinalEncoder(categories=order_mapping), [Columns.SPECIFIC_HOLIDAY, Columns.MONTH]),
    ("drop", [Columns.INFO_PAGE_TIME, Columns.PRODUCT_PAGE_TIME, Columns.GOOGLE_ANALYTICS_ER]),
    (RobustScaler(), make_column_selector(dtype_include="number"))
)

In [20]:
knn_classifier = KNeighborsClassifier(
    n_jobs=ModelConstants.N_JOBS
)

In [21]:
smote_sampling = SMOTE(random_state=ModelConstants.RANDOM_STATE)
adasyn_sampling = ADASYN(random_state=ModelConstants.RANDOM_STATE)

In [22]:
near_miss_sampling = NearMiss()
tomek_link_sampling = TomekLinks()

In [25]:
select_best_features = SelectKBest(score_func=mutual_info_classif)

In [26]:
column_transforms_only_model = Pipeline(
    [
        ("column_transformation", log_and_ordinal_transforms),
        ("over_sampling", smote_sampling),
        ("under_sampling", near_miss_sampling),
        ("feature_selection", select_best_features),
        ("knn", knn_classifier)
    ]
)

In [29]:
column_transformation_tune = {
    "column_transformation": [
        log_and_ordinal_transforms,
        norm_and_one_hot_transforms,
        power_transforms,
        robust_transformation,
    ]
}
over_sampling_tuning = {
    "over_sampling": [
        smote_sampling,
        adasyn_sampling,
    ]
}
over_sampling__sampling_strategy_tuning = {
    "over_sampling__sampling_strategy": [
        ResamplingStrategy.MINORITY_ONLY,
        ResamplingStrategy.ALL,
        0.2,
        0.3,
        0.4,
        0.5,
        0.6,
        0.7
    ],
}
under_sampling_tuning = {
    "under_sampling": [
        near_miss_sampling,
        tomek_link_sampling,
        "passthrough"
    ],
}
knn__n_neighbors_tuning = {
    "knn__n_neighbors": [5, 7, 9, 11, 13, 15, 17, 19, 21, 25]
}
knn__weights = {"knn__weights": ["uniform", "distance"],
}

knn_power = {
    "knn__p": [1, 2]
}

knn_feature_selection = {
    "feature_selection__k": [5, 7, 9, 10, 12, 14, 16, 18]
}

tuning_params_1 = (
        column_transformation_tune |
        over_sampling_tuning |
        over_sampling__sampling_strategy_tuning |
        under_sampling_tuning |
        knn__n_neighbors_tuning |
        knn__weights |
        knn_power |
        knn_feature_selection
)

tuning_params_2 = (
        column_transformation_tune |
        under_sampling_tuning |
        knn__n_neighbors_tuning |
        knn__weights |
        knn_power |
        knn_feature_selection
)

In [30]:
best_model = (
    TuneHyperParams()
    .random_grid_search(
     column_transforms_only_model,
     [tuning_params_1, tuning_params_2]
    )
    .fit_model(
        data.TRAINING.predictors,
        data.TRAINING.outcome
    )
    .get_best_model()
)

{   'column_transformation': ColumnTransformer(transformers=[('onehotencoder', OneHotEncoder(),
                                 ['Customer_Type']),
                                ('ordinalencoder',
                                 OrdinalEncoder(categories=[['0', '0.2', '0.4',
                                                             '0.6', '0.8',
                                                             '1'],
                                                            [1, 2, 3, 4, 5, 6,
                                                             7, 8, 9, 10, 11,
                                                             12]]),
                                 ['SpecificHoliday', 'Month']),
                                ('drop', 'drop',
                                 ['Info_Page_Time', 'ProductPage_Time',
                                  'GoogleAnalytics_ER']),
                                ('robustscaler', RobustScaler(),
                                 <sklearn.com

In [31]:
final_model_knn = FinalModelPerformance(
    model=best_model,
    data=data
)

final_model_knn.get_cross_validation_results()

,Metric for Training Set,Score
0,fit_time mean score,0.792414
1,score_time mean score,0.123304
2,5-fold CV F1 Score mean score,0.648334
3,5-fold CV Accuracy mean score,0.878328
4,5-fold CV Balanced Accuracy mean score,0.813104
5,5-fold CV Matthew's Correlation Coefficient me...,0.579898


In [32]:
final_model_knn.get_final_model_performance()

,Metric for Testing Set,Score
0,F1 Score,0.681983
1,Accuracy,0.892257
2,Balanced Accuracy,0.791804
3,Matthew's Correlation Coefficient,0.620185


In [33]:
dump(final_model_knn.model, "./models_exports/knn_classifier.joblib")

['./models_exports/knn_classifier.joblib']